In [ ]:
import ee
import datetime
import pandas as pd
import time
from datetime import datetime

In [ ]:
# Authenticating and initializing the Earth Engine API
ee.Authenticate()
ee.Initialize(project = "")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/Data 298B Project Data/Rice Image Datasets - with Location and Time/image_metadata_taiwan_filtered.csv")
df = pd.read_csv("/content/drive/MyDrive/Data 298B Project Data/Rice Image Datasets - with Location and Time/Rice Leaf Diseases - Taiwan Filtered/image_metadata_taiwan_filtered_location.csv")

In [ ]:
df.head()

,Id,Latitude,Longitude,Date,Class,Date and Time
0,P_20181227_153331_vHDR_Auto.jpg,24.073258,120.661451,2018-12-27,Brown Spot,2018:12:27 15:33:31
1,P_20181227_153343_vHDR_Auto (1).jpg,24.073258,120.661451,2018-12-27,Brown Spot,2018:12:27 15:33:43
2,P_20181227_153711_vHDR_Auto.jpg,24.073297,120.661364,2018-12-27,Brown Spot,2018:12:27 15:37:11
3,P_20181227_153709_vHDR_Auto.jpg,24.073297,120.661364,2018-12-27,Brown Spot,2018:12:27 15:37:09
4,P_20181227_154446_vHDR_Auto (1).jpg,24.074350,120.661598,2018-12-27,Brown Spot,2018:12:27 15:44:46


In [ ]:
'''
# Converting the datetime string to a pandas datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%Y:%m:%d %H:%M:%S')

# Extracting the date part and assigning it to a new column
df['Date_Only'] = df['Date'].dt.date

# Converting the date column to a string type in the 'YYYY-MM-DD' format
df['Date_Only'] = df['Date_Only'].astype(str)
'''

In [ ]:
df.head()

,Id,Latitude,Longitude,Date,Class,Date_Only
0,P_20181227_153331_vHDR_Auto.jpg,24.073258,120.661451,2018-12-27 15:33:31,Brown Spot,2018-12-27
1,P_20181227_153343_vHDR_Auto (1).jpg,24.073258,120.661451,2018-12-27 15:33:43,Brown Spot,2018-12-27
2,P_20181227_153711_vHDR_Auto.jpg,24.073297,120.661364,2018-12-27 15:37:11,Brown Spot,2018-12-27
3,P_20181227_153709_vHDR_Auto.jpg,24.073297,120.661364,2018-12-27 15:37:09,Brown Spot,2018-12-27
4,P_20181227_154446_vHDR_Auto (1).jpg,24.074350,120.661598,2018-12-27 15:44:46,Brown Spot,2018-12-27


In [ ]:
#df = pd.read_csv("/content/drive/MyDrive/Data 298B Project Data/Rice Image Datasets - with Location and Time/image_metadata_taiwan_filtered.csv")

# List which contains tuples which contain the Latitude, Longitude, and Date for each row in the dataframe
coordinates_and_dates = []

for _, row in df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    date_created = row['Date']
    coordinates_and_dates.append((lat, lon, date_created))

In [ ]:
coordinates_and_dates_small = coordinates_and_dates[:2]
print(coordinates_and_dates_small)

[(24.07325847222222, 120.66145113888888, '2018-12-27'), (24.07325847222222, 120.66145113888888, '2018-12-27')]


In [ ]:
print(len(coordinates_and_dates))

829


In [ ]:
# Getting rid of duplicate tuples

from collections import OrderedDict

# Using an OrderedDict to remove duplicates while maintaining order.
unique_coordinates_and_dates_ordered = list(OrderedDict.fromkeys(coordinates_and_dates))

print(unique_coordinates_and_dates_ordered)

[(24.07325847222222, 120.66145113888888, '2018-12-27'), (24.073297, 120.66136430555557, '2018-12-27'), (24.074350222222225, 120.66159827777778, '2018-12-27'), (24.074811472222223, 120.660849, '2018-12-27'), (24.0743365, 120.66161158333334, '2018-12-27'), (24.074484194444445, 120.66075047222223, '2018-12-27'), (24.074605916666663, 120.66084258333332, '2018-12-27'), (24.074743416666667, 120.66083911111112, '2018-12-27'), (24.078364, 120.65095230555556, '2018-12-27'), (24.07834108333333, 120.65094219444444, '2018-12-27'), (24.07832025, 120.65095430555556, '2018-12-27'), (24.071476222222223, 120.6508911388889, '2018-12-27'), (24.07146513888889, 120.65071144444444, '2018-12-27'), (24.069349222222225, 120.65324747222223, '2018-12-27'), (24.069375138888887, 120.65321777777778, '2018-12-27'), (24.066757944444443, 120.65322747222224, '2018-12-27'), (23.40822777777777, 120.31804722222222, '2019-03-15'), (23.40823333333333, 120.31806944444445, '2019-03-15'), (23.40823333333333, 120.31807777777776

In [ ]:
print(len(unique_coordinates_and_dates_ordered))

591


In [ ]:
# MODIS Satellite (latest)

import csv

def get_modis_values(latitude, longitude, date_str):
    point = ee.Geometry.Point([longitude, latitude])
    target_date = ee.Date(date_str)

    modis = ee.ImageCollection('MODIS/006/MOD13Q1')\
             .filterBounds(point)\
             .filterDate('2000-01-01', target_date)\
             .select(['NDVI', 'EVI'])\
             .sort('system:time_start', False)\
             .limit(3)  # Getting only the three most recent images

    ndvi_values = []
    evi_values = []

    for i in range(3):
        image = modis.toList(3).get(i)
        image = ee.Image(image)

        # Directly using the pre-calculated NDVI and EVI, scaled to actual values
        ndvi_val = image.select('NDVI').multiply(0.0001).reduceRegion(ee.Reducer.first(), point, 250).get('NDVI').getInfo()
        evi_val = image.select('EVI').multiply(0.0001).reduceRegion(ee.Reducer.first(), point, 250).get('EVI').getInfo()

        ndvi_values.append(ndvi_val)
        evi_values.append(evi_val)

    # Padding missing values for NDVI and EVI if fewer than 3 images are available
    while len(ndvi_values) < 3:
        ndvi_values.append(None)
    while len(evi_values) < 3:
        evi_values.append(None)

    # Combining NDVI and EVI values for the CSV
    values_list = [latitude, longitude, date_str] + ndvi_values + evi_values

    return values_list

def write_to_csv(filename, data):
    headers = ["Latitude", "Longitude", "Date",
               "NDVI MODIS", "NDVI - 1 MODIS", "NDVI - 2 MODIS",
               "EVI MODIS", "EVI - 1 MODIS", "EVI - 2 MODIS"]

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)


# Fetching MODIS values for each location and date
results = [get_modis_values(lat, lon, date) for lat, lon, date in unique_coordinates_and_dates_ordered]

csv_file_path = '/content/drive/MyDrive/Data 298B Project Data/Rice Image Datasets - with Location and Time/Rice Leaf Diseases - Taiwan Filtered/remote_sensing_modis_taiwan.csv'
write_to_csv(csv_file_path, results)

print(f"CSV file created: {csv_file_path}")


CSV file created: /content/drive/MyDrive/Data 298B Project Data/Rice Image Datasets - with Location and Time/Rice Leaf Diseases - Taiwan Filtered/remote_sensing_modis_taiwan.csv
